# **Train-Validate-Test Split**
The purpose of this notebook is to randomly sample the entire set of observations (9720) into a 60-10-30 train-validate-test set. Data can be taken from the validate or test and added ot the train if need be. Random sampling will happen on a player by player level taking 60%, 10%, 30% from each specific player

## **1.0: Introduction**
### **1.1: Data and Module Load** 

In [1]:
import pandas as pd
import numpy as np
import sklearn

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
feature_set_whole = pd.read_csv('/Users/alexiainman/Documents/Big Data Bowl/Data/Processed/tackles_feature_set_1.csv')

In [3]:
feature_set_whole.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9720 entries, 0 to 9719
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gameId            9720 non-null   int64  
 1   playId            9720 non-null   int64  
 2   nflId             9720 non-null   int64  
 3   tackle            9720 non-null   int64  
 4   assist            9720 non-null   int64  
 5   forcedFumble      9720 non-null   int64  
 6   pff_missedTackle  9720 non-null   int64  
 7   game_play_Id      9720 non-null   object 
 8   event_type        9720 non-null   float64
 9   s_avg_05          9720 non-null   float64
 10  s_avg_1           9720 non-null   float64
 11  bc_s_avg_05       9720 non-null   float64
 12  bc_s_avg_1        9720 non-null   float64
 13  cos_avg_05        9720 non-null   float64
 14  cos_avg_1         9720 non-null   float64
 15  do_avg_05         9720 non-null   float64
 16  do_avg_1          9720 non-null   float64


In [5]:
pd.set_option('display.max_columns', None)
feature_set_whole.head(50)

,gameId,playId,nflId,tackle,assist,forcedFumble,pff_missedTackle,game_play_Id,event_type,s_avg_05,s_avg_1,bc_s_avg_05,bc_s_avg_1,cos_avg_05,cos_avg_1,do_avg_05,do_avg_1,bc_do_avg_05,bc_do_avg_1,cos_05_null,cos_1_null,ballCarrierId
0,2022090800,101,42816,1,0,0,0,2022090800_101,1.0,3.520,4.332,5.694,6.830,0.428826,-0.385109,0.832711,0.645022,0.691600,0.929311,0,0,47857
1,2022090800,393,46232,1,0,0,0,2022090800_393,1.0,2.164,1.706,0.298,0.336,0.985862,0.787554,0.287244,0.604378,-0.093511,-0.303400,0,0,47853
2,2022090800,486,40166,1,0,0,0,2022090800_486,1.0,2.332,2.274,3.000,2.376,0.995057,0.993806,-0.236111,0.522356,-0.263867,0.533244,0,0,44881
3,2022090800,646,47939,1,0,0,0,2022090800_646,1.0,4.274,5.334,4.364,5.720,0.975704,0.987113,0.287667,0.059622,-0.590022,0.458689,0,0,47879
4,2022090800,818,40107,1,0,0,0,2022090800_818,1.0,0.854,1.828,1.448,2.134,0.191356,0.563645,0.215956,0.442644,-0.301644,-0.653667,0,0,47853
5,2022090800,1736,44976,1,0,0,0,2022090800_1736,1.0,4.794,4.612,4.398,4.650,0.124383,-0.728045,0.702711,0.598533,0.302244,0.916533,0,0,52460
6,2022090800,2860,46085,1,0,0,0,2022090800_2860,1.0,2.492,3.170,3.390,2.560,0.760174,-0.071476,0.523244,0.854733,0.882800,0.675067,0,0,44881
7,2022090800,3145,52647,1,0,0,0,2022090800_3145,1.0,2.526,1.584,2.538,1.270,0.758493,0.166756,0.320667,0.658911,-0.696578,-0.292800,0,0,47853
8,2022090800,3362,46190,1,0,0,0,2022090800_3362,1.0,4.638,4.050,3.230,4.426,-0.812444,-0.984762,0.184089,0.618489,0.726556,0.890378,0,0,47853
9,2022090800,3383,54488,1,0,0,0,2022090800_3383,1.0,2.106,1.668,2.210,1.382,0.992764,0.930209,0.496000,0.318089,-0.221467,-0.341511,0,0,44881


## **2.0: Feature Set Exploration**

### **2.1: Checking Prevalence of Players in Feature Set**

In [8]:
feature_set_whole.nflId.nunique()

760

In [7]:
feature_set_whole.ballCarrierId.nunique()

464

In [9]:
feature_set_whole.nflId.value_counts()

nflId
46269    63
46304    61
52435    59
41243    56
47913    51
         ..
53053     1
52535     1
54705     1
54665     1
53531     1
Name: count, Length: 760, dtype: int64

In [11]:
nflIds = feature_set_whole.nflId.value_counts()

In [13]:
count = sum(nflIds > 27)
print(count)


94


In [10]:
feature_set_whole.ballCarrierId.value_counts()

ballCarrierId
43334    186
47807    168
54572    168
46071    158
46104    154
        ... 
43496      1
48657      1
37118      1
48576      1
54663      1
Name: count, Length: 464, dtype: int64

## **3.0: Splitting Data**

### **3.1: Adding Target Label: Tackle_Status**

In [16]:
feature_set_whole['tackle_status'] = feature_set_whole['event_type'].apply(lambda x: 1 if x in [1, 3] else 0 if x in [2, 4] else np.nan)

In [17]:
feature_set_whole.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9720 entries, 0 to 9719
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gameId            9720 non-null   int64  
 1   playId            9720 non-null   int64  
 2   nflId             9720 non-null   int64  
 3   tackle            9720 non-null   int64  
 4   assist            9720 non-null   int64  
 5   forcedFumble      9720 non-null   int64  
 6   pff_missedTackle  9720 non-null   int64  
 7   game_play_Id      9720 non-null   object 
 8   event_type        9720 non-null   float64
 9   s_avg_05          9720 non-null   float64
 10  s_avg_1           9720 non-null   float64
 11  bc_s_avg_05       9720 non-null   float64
 12  bc_s_avg_1        9720 non-null   float64
 13  cos_avg_05        9720 non-null   float64
 14  cos_avg_1         9720 non-null   float64
 15  do_avg_05         9720 non-null   float64
 16  do_avg_1          9720 non-null   float64


In [18]:
feature_set_whole.tackle_status.value_counts()

tackle_status
1    7834
0    1886
Name: count, dtype: int64

In [19]:
feature_set_whole.event_type.value_counts()

event_type
1.0    7775
2.0    1859
3.0      59
4.0      27
Name: count, dtype: int64

---

### **3.2: Creating 60-10-30 Split**

In [39]:
# Set the random seed for reproducibility
np.random.seed(13)

# Create an empty DataFrame for each set
train = pd.DataFrame()
validation = pd.DataFrame()
test = pd.DataFrame()

# Group the data by 'nflId'
groups = feature_set_whole.groupby('nflId')

# For each group, randomly sample 30% of the data for the test set, 10% for the validation set, and the rest for the training set
for name, group in groups:
    total = group.shape[0]
    test_size = round(total * 0.3)
    validation_size = round(total * 0.1)
    train_size = total - test_size - validation_size
    
    test_group = group.sample(n=test_size)
    remaining_group = group.drop(test_group.index)
    
    validation_group = remaining_group.sample(n=validation_size)
    remaining_group = remaining_group.drop(validation_group.index)
    
    train_group = remaining_group.sample(n=train_size)
    
    test = pd.concat([test, test_group])
    validation = pd.concat([validation, validation_group])
    train = pd.concat([train, train_group])

In [42]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2917 entries, 3956 to 1872
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gameId            2917 non-null   int64  
 1   playId            2917 non-null   int64  
 2   nflId             2917 non-null   int64  
 3   tackle            2917 non-null   int64  
 4   assist            2917 non-null   int64  
 5   forcedFumble      2917 non-null   int64  
 6   pff_missedTackle  2917 non-null   int64  
 7   game_play_Id      2917 non-null   object 
 8   event_type        2917 non-null   float64
 9   s_avg_05          2917 non-null   float64
 10  s_avg_1           2917 non-null   float64
 11  bc_s_avg_05       2917 non-null   float64
 12  bc_s_avg_1        2917 non-null   float64
 13  cos_avg_05        2917 non-null   float64
 14  cos_avg_1         2917 non-null   float64
 15  do_avg_05         2917 non-null   float64
 16  do_avg_1          2917 non-null   float64
 1

### **3.3: Data Exploration of the Split Groups**

#### **3.3.0: Entire Feature Set**

In [45]:
feature_set_whole.tackle_status.value_counts(normalize=True)*100

tackle_status
1    80.596708
0    19.403292
Name: proportion, dtype: float64

In [50]:
###Checking number of Fred Warner observations in each set
feature_set_whole['nflId'].value_counts()[46139]

31

In [54]:
###Checking number of Joe Mixon observations in each set
feature_set_whole['ballCarrierId'].value_counts()[44860]

140

In [58]:
###Checking the number of Rashaad Penny observations in each set
feature_set_whole['ballCarrierId'].value_counts()[46096]

53

#### **3.3.1: Train**

In [66]:
train.tackle_status.value_counts()

tackle_status
1    4746
0    1110
Name: count, dtype: int64

In [44]:
train.tackle_status.value_counts(normalize=True) * 100


tackle_status
1    81.045082
0    18.954918
Name: proportion, dtype: float64

In [49]:
###Checking number of Fred Warner observations in each set
train['nflId'].value_counts()[46139]


19

In [55]:
###Checking number of Joe Mixon observations in each set
train['ballCarrierId'].value_counts()[44860]

78

In [59]:
###Checking the number of Rashaad Penny observations in each set
train['ballCarrierId'].value_counts()[46096]

31

#### **3.3.2: Validation**

In [46]:
validation.tackle_status.value_counts(normalize=True) * 100

tackle_status
1    77.930306
0    22.069694
Name: proportion, dtype: float64

In [51]:
###Checking number of Fred Warner observations in each set
validation['nflId'].value_counts()[46139]

3

In [56]:
###Checking number of Joe Mixon observations in each set
validation['ballCarrierId'].value_counts()[44860]

8

In [60]:
###Checking the number of Rashaad Penny observations in each set
validation['ballCarrierId'].value_counts()[46096]

5

#### **3.3: Test**

In [48]:
test.tackle_status.value_counts()

tackle_status
1    2350
0     567
Name: count, dtype: int64

In [47]:
test.tackle_status.value_counts(normalize=True) * 100

tackle_status
1    80.562221
0    19.437779
Name: proportion, dtype: float64

In [52]:
###Checking number of Fred Warner observations in each set
test['nflId'].value_counts()[46139]

9

In [57]:
###Checking number of Joe Mixon observations in each set
test['ballCarrierId'].value_counts()[44860]

54

In [61]:
###Checking the number of Rashaad Penny observations in each set
test['ballCarrierId'].value_counts()[46096]

17

## **4.0: Saving DFs for Modeling**

### **4.1: Train**

In [69]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5856 entries, 8593 to 1708
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gameId            5856 non-null   int64  
 1   playId            5856 non-null   int64  
 2   nflId             5856 non-null   int64  
 3   tackle            5856 non-null   int64  
 4   assist            5856 non-null   int64  
 5   forcedFumble      5856 non-null   int64  
 6   pff_missedTackle  5856 non-null   int64  
 7   game_play_Id      5856 non-null   object 
 8   event_type        5856 non-null   float64
 9   s_avg_05          5856 non-null   float64
 10  s_avg_1           5856 non-null   float64
 11  bc_s_avg_05       5856 non-null   float64
 12  bc_s_avg_1        5856 non-null   float64
 13  cos_avg_05        5856 non-null   float64
 14  cos_avg_1         5856 non-null   float64
 15  do_avg_05         5856 non-null   float64
 16  do_avg_1          5856 non-null   float64
 1

In [63]:
train_target = train['tackle_status']

In [67]:
train_features = train.drop(['tackle_status', 'event_type', 'nflId', 'ballCarrierId', 'gameId', 'playId', 'game_play_Id', 'tackle', 'assist', 'forcedFumble', 'pff_missedTackle'], axis=1)

In [70]:
train.to_csv('/Users/alexiainman/Documents/Big Data Bowl/Data/Processed/train.csv', index=False)

In [71]:
train_target.to_csv('/Users/alexiainman/Documents/Big Data Bowl/Data/Modeling_Data/train_target.csv', index=False)

In [72]:
train_features.to_csv('/Users/alexiainman/Documents/Big Data Bowl/Data/Modeling_Data/train_features.csv', index=False)

### **4.2: Validation**

In [73]:
validation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 947 entries, 1923 to 2632
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gameId            947 non-null    int64  
 1   playId            947 non-null    int64  
 2   nflId             947 non-null    int64  
 3   tackle            947 non-null    int64  
 4   assist            947 non-null    int64  
 5   forcedFumble      947 non-null    int64  
 6   pff_missedTackle  947 non-null    int64  
 7   game_play_Id      947 non-null    object 
 8   event_type        947 non-null    float64
 9   s_avg_05          947 non-null    float64
 10  s_avg_1           947 non-null    float64
 11  bc_s_avg_05       947 non-null    float64
 12  bc_s_avg_1        947 non-null    float64
 13  cos_avg_05        947 non-null    float64
 14  cos_avg_1         947 non-null    float64
 15  do_avg_05         947 non-null    float64
 16  do_avg_1          947 non-null    float64
 17

In [74]:
validation_target = validation['tackle_status']

In [77]:
validation_features = validation.drop(['tackle_status', 'event_type', 'nflId', 'ballCarrierId', 'gameId', 'playId', 'game_play_Id', 'tackle', 'assist', 'forcedFumble', 'pff_missedTackle'], axis=1)

In [79]:
validation.to_csv('/Users/alexiainman/Documents/Big Data Bowl/Data/Processed/validation.csv', index=False)

In [80]:
validation_target.to_csv('/Users/alexiainman/Documents/Big Data Bowl/Data/Modeling_Data/validation_target.csv', index=False)

In [81]:
validation_features.to_csv('/Users/alexiainman/Documents/Big Data Bowl/Data/Modeling_Data/validation_features.csv', index=False)

### **4.3: Test**

In [82]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2917 entries, 3956 to 1872
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gameId            2917 non-null   int64  
 1   playId            2917 non-null   int64  
 2   nflId             2917 non-null   int64  
 3   tackle            2917 non-null   int64  
 4   assist            2917 non-null   int64  
 5   forcedFumble      2917 non-null   int64  
 6   pff_missedTackle  2917 non-null   int64  
 7   game_play_Id      2917 non-null   object 
 8   event_type        2917 non-null   float64
 9   s_avg_05          2917 non-null   float64
 10  s_avg_1           2917 non-null   float64
 11  bc_s_avg_05       2917 non-null   float64
 12  bc_s_avg_1        2917 non-null   float64
 13  cos_avg_05        2917 non-null   float64
 14  cos_avg_1         2917 non-null   float64
 15  do_avg_05         2917 non-null   float64
 16  do_avg_1          2917 non-null   float64
 1

In [83]:
test_target = test['tackle_status']

In [86]:
test_features = test.drop(['tackle_status', 'event_type', 'gameId', 'playId', 'game_play_Id', 'tackle', 'assist', 'forcedFumble', 'pff_missedTackle'], axis=1)

In [88]:
test.to_csv('/Users/alexiainman/Documents/Big Data Bowl/Data/Processed/test.csv', index=False)

In [89]:
test_target.to_csv('/Users/alexiainman/Documents/Big Data Bowl/Data/Modeling_Data/test_target.csv', index=False)

In [90]:
test_features.to_csv('/Users/alexiainman/Documents/Big Data Bowl/Data/Modeling_Data/test_features.csv', index=False)